<a href="https://colab.research.google.com/github/RAGHVI27/Deep-learning-based-Chronic-Kidney-Disease-Detection/blob/main/CSE_4020_JCOMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer


df = pd.read_csv('kidney_disease_1.csv')
#df = df.replace(r'^\s*$', np.nan, regex=True)
# Data cleaning and preprocessing
numerical_columns = ['age', 'bp', 'sg', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc']
#df[numerical_columns] = df[numerical_columns].astype(float)
#imputer = SimpleImputer(strategy='mean')
#df[numerical_columns] = imputer.fit_transform(df[numerical_columns])
# Impute missing values in numerical columns with mode
numerical_imputer = SimpleImputer(strategy='most_frequent')
df[numerical_columns] = numerical_imputer.fit_transform(df[numerical_columns])
# Assuming df is your DataFrame
df.replace('\t?', np.nan, inplace=True)  # Replace '\t?' with NaN

for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')







In [2]:
# Replace missing values with the mode for categorical columns
categorical_columns = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']
for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [3]:
# Encode categorical features using Label Encoding
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Standardize numerical features
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Split the data into training and testing sets
X = df.drop('classification', axis=1)
y = df['classification']


# Impute missing values
X.fillna(X.mean(), inplace=True)  # You can use other imputation methods as well

# Convert target variable to numeric
y = pd.factorize(df['classification'])[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
"""
# Feature selection using Random Forest
X = df.drop('classification', axis=1)
y = df['classification']
"""


#############################MODIFICATION REQUIRED

# Feature selection using Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
sfm = SelectFromModel(rf, threshold=0.1)
sfm.fit(X_train, y_train)
X_train = sfm.transform(X_train)
X_test = sfm.transform(X_test)

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Assuming df is your DataFrame with NaN values
# Assuming 'classification' is the target variable, adjust accordingly

# Drop the target variable and any other non-numeric columns if needed
X = df.drop(['classification'], axis=1)

# Impute missing values
X.fillna(X.mean(), inplace=True)  # You can use other imputation methods as well

# Convert target variable to numeric
y = pd.factorize(df['classification'])[0]

# Create a Random Forest classifier
clf = RandomForestClassifier()

# Fit the classifier
clf.fit(X, y)

# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame with feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance DataFrame
print(feature_importance_df)


   Feature  Importance
0       id    0.341896
12      sc    0.115544
16     pcv    0.111346
3       sg    0.108398
15    hemo    0.083960
4       al    0.082725
19     htn    0.024672
18      rc    0.023718
13     sod    0.021059
11      bu    0.016115
20      dm    0.014729
10     bgr    0.014324
17      wc    0.010580
1      age    0.007730
14     pot    0.005840
2       bp    0.004091
22   appet    0.004010
23      pe    0.002048
7       pc    0.002042
5       su    0.001922
24     ane    0.001270
6      rbc    0.001254
8      pcc    0.000584
9       ba    0.000092
21     cad    0.000053


In [20]:
# LSTM Model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Train the LSTM model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Get the output of the LSTM model
lstm_output = model.layers[0](X_train)
# Evaluate the model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # Threshold for binary classification

from sklearn.metrics import precision_score, recall_score, f1_score


precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
recall = recall_score(y_test, y_pred, average='macro', zero_division=1)

# Print classification report
print(classification_report(y_test, y_pred))

Epoch 1/10
10/10 [==============================] - 4s 10ms/step - loss: 0.7393 - accuracy: 0.3313
Epoch 2/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5123 - accuracy: 0.0063
Epoch 3/10
10/10 [==============================] - 0s 13ms/step - loss: 0.3554 - accuracy: 0.0063
Epoch 4/10
10/10 [==============================] - 0s 11ms/step - loss: 0.1869 - accuracy: 0.0063
Epoch 5/10
10/10 [==============================] - 0s 15ms/step - loss: -0.1256 - accuracy: 0.0219
Epoch 6/10
10/10 [==============================] - 0s 16ms/step - loss: -0.7356 - accuracy: 0.1750
Epoch 7/10
10/10 [==============================] - 0s 10ms/step - loss: -1.6772 - accuracy: 0.3125
Epoch 8/10
10/10 [==============================] - 0s 10ms/step - loss: -2.6321 - accuracy: 0.4375
Epoch 9/10
10/10 [==============================] - 0s 10ms/step - loss: -3.6969 - accuracy: 0.5156
Epoch 10/10
10/10 [==============================] - 0s 9ms/step - loss: -4.2919 - accuracy: 0.5500


3/3 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        52
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00        28

    accuracy                           0.65        80
   macro avg       0.33      0.33      0.33        80
weighted avg       0.65      0.65      0.65        80



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [22]:
print("NaN values in y_pred:", np.isnan(y_pred).any())
print("Inf values in y_pred:", np.isinf(y_pred).any())


NaN values in y_pred: False
Inf values in y_pred: False


# New section

In [26]:
from keras.models import Sequential
from keras.layers import GRU, Dense

# GRU Model
gru_model = Sequential()
gru_model.add(GRU(64, input_shape=(lstm_output.shape[1], 1)))
gru_model.add(Dense(1, activation='sigmoid'))

gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Reshape data for GRU input
lstm_output_reshaped = lstm_output.reshape(lstm_output.shape[0], lstm_output.shape[1], 1)

# Train the GRU model
gru_model.fit(lstm_output_reshaped, y_train, epochs=10, batch_size=32)

# Get the output of the GRU model
gru_output = gru_model.layers[0](lstm_output_reshaped)

# Evaluate the GRU model
gru_predictions = gru_model.predict(lstm_output_reshaped)
gru_predictions = (gru_predictions > 0.5)  # Threshold for binary classification

# Print classification report for the GRU model
print(classification_report(y_train, gru_predictions))


Epoch 1/10
10/10 [==============================] - 5s 49ms/step - loss: 0.6618 - accuracy: 0.0625
Epoch 2/10
10/10 [==============================] - 1s 55ms/step - loss: 0.5772 - accuracy: 0.0063
Epoch 3/10
10/10 [==============================] - 1s 63ms/step - loss: 0.4316 - accuracy: 0.0063
Epoch 4/10
10/10 [==============================] - 0s 38ms/step - loss: 0.1268 - accuracy: 0.0063
Epoch 5/10
10/10 [==============================] - 0s 38ms/step - loss: -0.4075 - accuracy: 0.0063
Epoch 6/10
10/10 [==============================] - 0s 37ms/step - loss: -1.2383 - accuracy: 0.0063
Epoch 7/10
10/10 [==============================] - 0s 35ms/step - loss: -2.7263 - accuracy: 0.4875
Epoch 8/10
10/10 [==============================] - 0s 37ms/step - loss: -3.5896 - accuracy: 0.5969
Epoch 9/10
10/10 [==============================] - 0s 33ms/step - loss: -4.0986 - accuracy: 0.6062
Epoch 10/10
10/10 [==============================] - 0s 33ms/step - loss: -4.5691 - accuracy: 0.6062


10/10 [==============================] - 1s 11ms/step
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       196
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00       122

    accuracy                           0.60       320
   macro avg       0.33      0.32      0.33       320
weighted avg       0.61      0.60      0.60       320



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from keras.layers import SimpleRNN

# RNN Model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(32, input_shape=(gru_output.shape[1], 1)))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Reshape data for RNN input
gru_output_reshaped = gru_output.reshape(gru_output.shape[0], gru_output.shape[1], 1)

# Train the RNN model
rnn_model.fit(gru_output_reshaped, y_train, epochs=10, batch_size=32)

# Get the output of the RNN model
rnn_predictions = rnn_model.layers[0](gru_output_reshaped)

# Evaluate the RNN model
rnn_predictions = rnn_model.predict(gru_output_reshaped)
rnn_predictions = (rnn_predictions > 0.5)  # Threshold for binary classification

# Print classification report for the RNN model
print(classification_report(y_train, rnn_predictions))


Epoch 1/10
10/10 [==============================] - 2s 21ms/step - loss: -0.2006 - accuracy: 0.6062
Epoch 2/10
10/10 [==============================] - 0s 16ms/step - loss: -0.8624 - accuracy: 0.6062
Epoch 3/10
10/10 [==============================] - 0s 23ms/step - loss: -1.3212 - accuracy: 0.6062
Epoch 4/10
10/10 [==============================] - 0s 27ms/step - loss: -1.6666 - accuracy: 0.6062
Epoch 5/10
10/10 [==============================] - 0s 26ms/step - loss: -1.9635 - accuracy: 0.6062
Epoch 6/10
10/10 [==============================] - 0s 35ms/step - loss: -2.2388 - accuracy: 0.6062
Epoch 7/10
10/10 [==============================] - 0s 42ms/step - loss: -2.4763 - accuracy: 0.6094
Epoch 8/10
10/10 [==============================] - 0s 18ms/step - loss: -2.7017 - accuracy: 0.6062
Epoch 9/10
10/10 [==============================] - 0s 20ms/step - loss: -2.8950 - accuracy: 0.6062
Epoch 10/10
10/10 [==============================] - 0s 6ms/step
              precision    recall  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
 # now we apply svm as meta learner

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

# Get the output of the RNN model
rnn_output = rnn_model.layers[0](gru_output)

# Combine the RNN output with the original features
X_train_combined = pd.concat([X_train, pd.DataFrame(rnn_output, columns=['rnn_output'])], axis=1)
X_test_combined = pd.concat([X_test, pd.DataFrame(rnn_model.predict(X_test_gru_output), columns=['rnn_output'])], axis=1)

# Define and train the SVM meta-learner
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}
svm = SVC(random_state=42)

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_combined, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']

# Train the SVM classifier with the best hyperparameters
best_svm = SVC(kernel=best_kernel, C=best_C, random_state=42)
best_svm.fit(X_train_combined, y_train)

# Make predictions with the best SVM classifier
y_pred_best_svm = best_svm.predict(X_test_combined)

# Print classification report for the final combined model with the best SVM
print("Best Kernel:", best_kernel)
print("Best C:", best_C)
print(classification_report(y_test, y_pred_best_svm))
#svm.fit(X_train_combined, y_train)

# Make predictions with the SVM meta-learner
#y_pred_svm = svm.predict(X_test_combined)

# Print classification rport for the final combined model
#print(classification_report(y_test, y_pred_svm))


ValueError: ignored

In [48]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd
# Reshape X_train to 2D array
num_samples, sequence_length, num_features = X_train.shape
X_train_2d = X_train.reshape(num_samples, -1)

# Create a DataFrame
columns = [f'feature_{i}_time_{j}' for i in range(num_features) for j in range(sequence_length)]
df_X_train = pd.DataFrame(X_train_2d, columns=columns)

# Display the DataFrame
print(df_X_train)
# Assuming rnn_predictions contains the output from the RNN model

# Combine the RNN output with the original features
#X_train_combined = pd.concat([X_train, pd.DataFrame(rnn_predictions, columns=['rnn_output'])], axis=1)
#X_test_combined = pd.concat([X_test, pd.DataFrame(rnn_model.predict(X_test_rnn_output), columns=['rnn_output'])], axis=1)

# Convert rnn_predictions to a DataFrame
rnn_predictions_df = pd.DataFrame(rnn_predictions, columns=['rnn_output'])

# Convert X_train to a DataFrame if it's not already
#if not isinstance(X_train, pd.DataFrame):
    #X_train = pd.DataFrame(X_train, columns=['feature1', 'feature2', ...])  # Replace with your column names
#X_test_rnn_output = rnn_model.predict(X_test)
# Combine the RNN output with the original features
X_train_combined = pd.concat([df_X_train, rnn_predictions_df], axis=1)
X_test_combined = pd.concat([X_test, pd.DataFrame(rnn_model.predict(rnn_predictions_df), columns=['rnn_output'])], axis=1)

# Define and train the SVM meta-learner
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}
svm = SVC(random_state=42)

grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_combined, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_kernel = best_params['kernel']
best_C = best_params['C']

# Train the SVM classifier with the best hyperparameters
best_svm = SVC(kernel=best_kernel, C=best_C, random_state=42)
best_svm.fit(X_train_combined, y_train)

# Make predictions with the best SVM classifier
y_pred_best_svm = best_svm.predict(X_test_combined)

# Print classification report for the final combined model with the best SVM
print("Best Kernel:", best_kernel)
print("Best C:", best_C)
print(classification_report(y_test, y_pred_best_svm))


     feature_0_time_0  feature_0_time_1  feature_0_time_2  feature_0_time_3  \
0                 3.0         -2.342354          4.000000          0.143536   
1                18.0          1.342766          0.000000         -0.301054   
2               202.0          0.421486          1.016949         -0.016517   
3               250.0          1.342766          0.000000         -0.318837   
4               274.0          0.421486          0.000000         -0.407755   
..                ...               ...               ...               ...   
315              71.0         -1.421074          1.000000          0.054618   
316             106.0          0.421486          1.016949          0.552558   
317             270.0          1.342766          0.000000         -0.336621   
318             348.0          0.421486          0.000000         -0.443322   
319             102.0         -1.421074          0.000000         -0.158785   

     feature_0_time_4  
0           -0.885839  
1  

TypeError: ignored

In [35]:
print(X_train)



[[[ 3.00000000e+00]
  [-2.34235440e+00]
  [ 4.00000000e+00]
  [ 1.43535723e-01]
  [-8.85838942e-01]]

 [[ 1.80000000e+01]
  [ 1.34276560e+00]
  [ 0.00000000e+00]
  [-3.01053786e-01]
  [-2.75390345e-01]]

 [[ 2.02000000e+02]
  [ 4.21485600e-01]
  [ 1.01694915e+00]
  [-1.65165002e-02]
  [-1.86255670e+00]]

 ...

 [[ 2.70000000e+02]
  [ 1.34276560e+00]
  [ 0.00000000e+00]
  [-3.36620946e-01]
  [ 2.12968533e-01]]

 [[ 3.48000000e+02]
  [ 4.21485600e-01]
  [ 0.00000000e+00]
  [-4.43322428e-01]
  [ 5.79237692e-01]]

 [[ 1.02000000e+02]
  [-1.42107440e+00]
  [ 0.00000000e+00]
  [-1.58785143e-01]
  [ 1.55595545e+00]]]


In [37]:
import pandas as pd

# Reshape X_train to 2D array
num_samples, sequence_length, num_features = X_train.shape
X_train_2d = X_train.reshape(num_samples, -1)

# Create a DataFrame
columns = [f'feature_{i}_time_{j}' for i in range(num_features) for j in range(sequence_length)]
df_X_train = pd.DataFrame(X_train_2d, columns=columns)

# Display the DataFrame
print(df_X_train)


     feature_0_time_0  feature_0_time_1  feature_0_time_2  feature_0_time_3  \
0                 3.0         -2.342354          4.000000          0.143536   
1                18.0          1.342766          0.000000         -0.301054   
2               202.0          0.421486          1.016949         -0.016517   
3               250.0          1.342766          0.000000         -0.318837   
4               274.0          0.421486          0.000000         -0.407755   
..                ...               ...               ...               ...   
315              71.0         -1.421074          1.000000          0.054618   
316             106.0          0.421486          1.016949          0.552558   
317             270.0          1.342766          0.000000         -0.336621   
318             348.0          0.421486          0.000000         -0.443322   
319             102.0         -1.421074          0.000000         -0.158785   

     feature_0_time_4  
0           -0.885839  
1  

In [41]:
# Reshape X_train to 2D array
num_samples, sequence_length, num_features = X_train.shape
X_train_2d = X_train.reshape(num_samples, -1)
# Create a DataFrame
columns = [f'feature_{i}_time_{j}' for i in range(num_features) for j in range(sequence_length)]
df_X_train = pd.DataFrame(X_train_2d, columns=columns)
# Convert rnn_predictions to a DataFrame
rnn_predictions_df = pd.DataFrame(rnn_predictions, columns=['rnn_output'])

# Combine the RNN output with the original features
X_train_combined = pd.concat([df_X_train, rnn_predictions_df], axis=1)
# Convert rnn_model.predict(X_test_rnn_output) to a DataFrame
rnn_test_predictions_df = pd.DataFrame(rnn_model.predict(X_test_rnn_output), columns=['rnn_output'])

# Combine the RNN output with the original features in the test set
X_test_combined = pd.concat([X_test, rnn_test_predictions_df], axis=1)


TypeError: ignored

In [42]:
print(X_test_rnn_output.shape)


(80, 1)


In [43]:
# Example: Reshape X_test_rnn_output to have the correct shape
X_test_rnn_output_reshaped = X_test_rnn_output.reshape((num_samples, sequence_length, num_features))


ValueError: ignored